In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo")
from turing.utils import *
import turing.pinns as tu
import turing.pinns_multi_nodes as tu_multi
from turing.loss_functions import *

GPU = False
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-07-07 12:19:27.801286: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1892] Ignoring visible gpu device (device: 0, name: Quadro K2000, pci bus id: 0000:05:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.


In [12]:
class ASDM_multi(ASDM):
    def __init__(self,
                 dtype,
                 init_value = 10.0,
                 D_a = None,
                 D_s = None,
                 sigma_a = None,
                 sigma_s = None,
                 mu_a = None,
                 rho_a = None,
                 rho_s = None,
                 kappa_a = None,
                 print_precision=".5f"
                ):
        super().__init__(dtype, init_value, D_a, D_s, sigma_a, sigma_s, 
                         mu_a, rho_a, rho_s, kappa_a, print_precision)
    @tf.function    
    def loss(self, pinn, x):
        outputs, f_a, f_s = super().loss(pinn, x)
        return outputs, tf.concat([tf.expand_dims(f_a, axis=1), 
                          tf.expand_dims(f_s, axis=1)], axis = 1)

# Load data

In [13]:
data_path = os.path.abspath("turing.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("turing_t.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
T = t_star.shape[0]    
    
L = 50
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']


In [17]:
class Non_zero_params(Loss):
    def __init__(self,
                 loss_obj:Loss,
                 print_precision=".5f"):
        super().__init__(name=f"Non-zero params for {loss_obj.name}", print_precision=print_precision)
        self.loss_obj = loss_obj
        #self._trainables_ = loss_obj.trainables()
    
    @tf.function
    def loss(self, pinn, x):
        # Lennard-Jones potential
        params = tf.stack(self.loss_obj.trainables())
        epsilon = params.shape[0]
        twelve = 12*tf.ones_like(params, dtype=pinn.dtype)
        six = 6*tf.ones_like(params, dtype=pinn.dtype)
        return tf.reduce_sum((4e-6/epsilon)*(tf.pow(1e-6/(params+1e-10), twelve) - tf.pow(1e-6/(params+1e-10), six)))        
        


# Test

In [18]:
layers = [3, 64, 64, 64, 64, 2]

pinn = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss = ASDM_multi(dtype=tf.float64, D_a = 0.005, D_s = 0.2)
non_zero_loss = Non_zero_params(pde_loss)

In [21]:
model = tu_multi.TINN_multi_nodes(pinn, 
                                  pde_loss,
                                  extra_loss = [non_zero_loss, non_zero_loss], 
                                  nodes_n = 2,
                                  node_names= ["u", "v"],
                                  alpha = 0.25)

In [22]:
results = model.train(epochs = 2,
                      batch_size = 512,
                      X = obs_X,
                      Y = obs_Y,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 0.75465
total loss: 1732.67198, total regularisd loss: 1732.67198
obs u loss: 540.17930, pde u loss: 1.58507
obs v loss: 1005.33491, pde v loss: 185.57270
lambda obs u: 1.00000, lambda pde u: 1.00000
lambda obs v: 1.00000, lambda pde v: 1.00000
sigma_a: 10.02372 sigma_s: 5.74427 mu_a: 9.72403 rho_a: 10.38609 
rho_s: 14.28363 kappa_a: 5.03918 
extra loss Non-zero params for Loss_ASDM: -0.00000
extra loss Non-zero params for Loss_ASDM: -0.00000
Time taken: 38.41s

Start of epoch 1
Training observations acc over epoch: 0.43076
total loss: 954.62885, total regularisd loss: 954.62885
obs u loss: 383.32027, pde u loss: 0.40274
obs v loss: 498.87197, pde v loss: 72.03388
lambda obs u: 2.02126, lambda pde u: 0.25000
lambda obs v: 1.45772, lambda pde v: 0.27101
sigma_a: 9.67559 sigma_s: 5.33911 mu_a: 10.14208 rho_a: 9.92100 
rho_s: 14.63530 kappa_a: 4.56284 
extra loss Non-zero params for Loss_ASDM: -0.00000
extra loss Non-zero params for 

In [6]:
results = model.train(epochs = 500,
                      batch_size = 512,
                      X = obs_X,
                      Y = obs_Y,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 8.24665
total loss: 36901.80392, total regularisd loss: 36901.80392
obs u loss: 583.56867, pde u loss: 979.66713
lambda obs u: 1.00000, lambda pde u: 1.00000
obs v loss: 16305.55737, pde v loss: 19033.01075
lambda obs v: 1.00000, lambda pde v: 1.00000
sigma_a:0:10.02798, sigma_s:0:9.20963, mu_a:0:9.85684, rho_a:0:10.55152, rho_s:0:11.05547, kappa_a:0:8.90927
Time taken: 42.37s

Start of epoch 1
Training observations acc over epoch: 6.36216
total loss: 20312.88319, total regularisd loss: 20312.88319
obs u loss: 648.02180, pde u loss: 1.55567
lambda obs u: 0.31195, lambda pde u: 0.25000
obs v loss: 12381.69397, pde v loss: 7281.61175
lambda obs v: 2.47256, lambda pde v: 0.96549
sigma_a:0:10.07481, sigma_s:0:8.43107, mu_a:0:9.70449, rho_a:0:10.99664, rho_s:0:11.87508, kappa_a:0:7.99093
Time taken: 31.17s

Start of epoch 2
Training observations acc over epoch: 2.08168
total loss: 13723.22462, total regularisd loss: 18275.13640
obs u l

Training observations acc over epoch: 0.13252
total loss: 22935.60449, total regularisd loss: 978.24134
obs u loss: 247.85661, pde u loss: 51.33103
lambda obs u: 3.94069, lambda pde u: 0.00434
obs v loss: 23.55433, pde v loss: 14.50115
lambda obs v: 0.05385, lambda pde v: 0.00111
sigma_a:0:8.04979, sigma_s:0:4.33073, mu_a:0:10.86707, rho_a:0:10.03475, rho_s:0:15.37159, kappa_a:0:3.51150
Time taken: 25.82s

Start of epoch 21
Training observations acc over epoch: 0.13256
total loss: 25752.52506, total regularisd loss: 978.23050
obs u loss: 247.85711, pde u loss: 48.97580
lambda obs u: 3.94281, lambda pde u: 0.00416
obs v loss: 23.63260, pde v loss: 15.56295
lambda obs v: 0.05190, lambda pde v: 0.00112
sigma_a:0:8.01463, sigma_s:0:4.33809, mu_a:0:10.84376, rho_a:0:10.05874, rho_s:0:15.35966, kappa_a:0:3.49619
Time taken: 25.78s

Start of epoch 22
Training observations acc over epoch: 0.13254
total loss: 26361.61401, total regularisd loss: 978.54833
obs u loss: 247.81732, pde u loss: 49.68

Training observations acc over epoch: 0.13240
total loss: 26430.97725, total regularisd loss: 976.86850
obs u loss: 247.83450, pde u loss: 41.99728
lambda obs u: 3.93181, lambda pde u: 0.00372
obs v loss: 23.31726, pde v loss: 13.90712
lambda obs v: 0.06352, lambda pde v: 0.00095
sigma_a:0:7.17768, sigma_s:0:4.57222, mu_a:0:10.49081, rho_a:0:10.36027, rho_s:0:14.26607, kappa_a:0:2.96719
Time taken: 25.94s

Start of epoch 41
Training observations acc over epoch: 0.13235
total loss: 24005.29518, total regularisd loss: 975.87138
obs u loss: 247.78328, pde u loss: 38.58252
lambda obs u: 3.92892, lambda pde u: 0.00370
obs v loss: 23.27399, pde v loss: 12.51443
lambda obs v: 0.06650, lambda pde v: 0.00088
sigma_a:0:7.13985, sigma_s:0:4.59556, mu_a:0:10.46166, rho_a:0:10.38328, rho_s:0:14.18992, kappa_a:0:2.92683
Time taken: 25.82s

Start of epoch 42
Training observations acc over epoch: 0.13235
total loss: 26362.57176, total regularisd loss: 975.19939
obs u loss: 247.77533, pde u loss: 40.96

Training observations acc over epoch: 0.13229
total loss: 28905.81172, total regularisd loss: 974.30266
obs u loss: 247.76773, pde u loss: 34.46392
lambda obs u: 3.93078, lambda pde u: 0.00308
obs v loss: 23.15950, pde v loss: 15.11957
lambda obs v: 0.06522, lambda pde v: 0.00092
sigma_a:0:6.15025, sigma_s:0:4.85576, mu_a:0:10.12926, rho_a:0:10.56831, rho_s:0:12.87092, kappa_a:0:2.36605
Time taken: 26.21s

Start of epoch 61
Training observations acc over epoch: 0.13225
total loss: 25274.80909, total regularisd loss: 975.50881
obs u loss: 247.76046, pde u loss: 32.11516
lambda obs u: 3.92276, lambda pde u: 0.00321
obs v loss: 23.08331, pde v loss: 13.27004
lambda obs v: 0.07312, lambda pde v: 0.00092
sigma_a:0:6.10657, sigma_s:0:4.87587, mu_a:0:10.10629, rho_a:0:10.58245, rho_s:0:12.79257, kappa_a:0:2.33293
Time taken: 26.06s

Start of epoch 62
Training observations acc over epoch: 0.13224
total loss: 25552.04552, total regularisd loss: 973.66648
obs u loss: 247.74875, pde u loss: 33.63

Training observations acc over epoch: 0.13219
total loss: 26962.30335, total regularisd loss: 976.77383
obs u loss: 247.73145, pde u loss: 25.76422
lambda obs u: 3.93123, lambda pde u: 0.00253
obs v loss: 22.98634, pde v loss: 15.67529
lambda obs v: 0.06523, lambda pde v: 0.00100
sigma_a:0:5.04807, sigma_s:0:5.09405, mu_a:0:9.79839, rho_a:0:10.65646, rho_s:0:11.44695, kappa_a:0:1.85587
Time taken: 26.05s

Start of epoch 81
Training observations acc over epoch: 0.13217
total loss: 26341.52892, total regularisd loss: 975.44515
obs u loss: 247.72476, pde u loss: 26.91702
lambda obs u: 3.92591, lambda pde u: 0.00252
obs v loss: 22.95859, pde v loss: 15.30576
lambda obs v: 0.07055, lambda pde v: 0.00102
sigma_a:0:5.01608, sigma_s:0:5.12427, mu_a:0:9.75989, rho_a:0:10.68231, rho_s:0:11.35797, kappa_a:0:1.82033
Time taken: 26.38s

Start of epoch 82
Training observations acc over epoch: 0.13214
total loss: 24505.04688, total regularisd loss: 974.20725
obs u loss: 247.71567, pde u loss: 26.9008

...

Training observations acc over epoch: 0.00113
total loss: 4.09267, total regularisd loss: 3.48662
obs u loss: 1.39774, pde u loss: 0.95158
lambda obs u: 1.07643, lambda pde u: 0.73923
obs v loss: 0.90728, pde v loss: 0.27515
lambda obs v: 1.27739, lambda pde v: 0.90695
sigma_a:0:0.45797, sigma_s:0:0.01949, mu_a:0:2.36561, rho_a:0:1.99578, rho_s:0:0.02053, kappa_a:0:0.28437
Time taken: 27.64s

Start of epoch 401
Training observations acc over epoch: 0.00074
total loss: 3.76305, total regularisd loss: 3.19870
obs u loss: 0.77868, pde u loss: 1.62756
lambda obs u: 0.57815, lambda pde u: 1.48912
obs v loss: 0.73896, pde v loss: 0.23532
lambda obs v: 1.12532, lambda pde v: 0.80742
sigma_a:0:0.42614, sigma_s:0:0.01949, mu_a:0:2.29023, rho_a:0:1.94291, rho_s:0:0.01994, kappa_a:0:0.28303
Time taken: 27.37s

Start of epoch 402
Training observations acc over epoch: 0.00109
total loss: 4.05254, total regularisd loss: 3.28861
obs u loss: 1.36450, pde u loss: 0.87569
lambda obs u: 1.06435, lambda p

Training observations acc over epoch: 0.00026
total loss: 0.75070, total regularisd loss: 0.62226
obs u loss: 0.37334, pde u loss: 0.02440
lambda obs u: 1.13591, lambda pde u: 0.31432
obs v loss: 0.15296, pde v loss: 0.09447
lambda obs v: 0.58689, lambda pde v: 1.96289
sigma_a:0:0.01879, sigma_s:0:0.01763, mu_a:0:0.11360, rho_a:0:0.09629, rho_s:0:0.01786, kappa_a:0:0.26469
Time taken: 27.78s

Start of epoch 421
Training observations acc over epoch: 0.00025
total loss: 0.65662, total regularisd loss: 0.68874
obs u loss: 0.36961, pde u loss: 0.01042
lambda obs u: 1.23539, lambda pde u: 0.12328
obs v loss: 0.14768, pde v loss: 0.09117
lambda obs v: 0.58782, lambda pde v: 2.05352
sigma_a:0:0.01201, sigma_s:0:0.01798, mu_a:0:0.07890, rho_a:0:0.06894, rho_s:0:0.01829, kappa_a:0:0.27655
Time taken: 27.32s

Start of epoch 422
Training observations acc over epoch: 0.00023
total loss: 0.59240, total regularisd loss: 0.66768
obs u loss: 0.32577, pde u loss: 0.00586
lambda obs u: 1.22436, lambda p

Training observations acc over epoch: 0.00018
total loss: 1.19090, total regularisd loss: 0.53223
obs u loss: 0.26179, pde u loss: 0.00070
lambda obs u: 1.18340, lambda pde u: 0.00094
obs v loss: 0.10467, pde v loss: 0.07352
lambda obs v: 0.47334, lambda pde v: 2.34232
sigma_a:0:0.00000, sigma_s:0:0.01738, mu_a:0:0.01003, rho_a:0:0.01047, rho_s:0:0.01778, kappa_a:0:0.29963
Time taken: 27.18s

Start of epoch 441
Training observations acc over epoch: 0.00017
total loss: 1.20057, total regularisd loss: 0.51373
obs u loss: 0.24883, pde u loss: 0.00070
lambda obs u: 1.14795, lambda pde u: 0.00098
obs v loss: 0.10485, pde v loss: 0.07242
lambda obs v: 0.50521, lambda pde v: 2.34587
sigma_a:0:0.00005, sigma_s:0:0.01727, mu_a:0:0.01002, rho_a:0:0.01063, rho_s:0:0.01777, kappa_a:0:0.30151
Time taken: 26.63s

Start of epoch 442
Training observations acc over epoch: 0.00017
total loss: 1.17385, total regularisd loss: 0.51152
obs u loss: 0.25194, pde u loss: 0.00070
lambda obs u: 1.17452, lambda p

Training observations acc over epoch: 0.00015
total loss: 1.24594, total regularisd loss: 0.45391
obs u loss: 0.22199, pde u loss: 0.00069
lambda obs u: 1.13342, lambda pde u: 0.00100
obs v loss: 0.09300, pde v loss: 0.06747
lambda obs v: 0.50569, lambda pde v: 2.35989
sigma_a:0:0.00000, sigma_s:0:0.01690, mu_a:0:0.01007, rho_a:0:0.01034, rho_s:0:0.01784, kappa_a:0:0.28739
Time taken: 27.06s

Start of epoch 461
Training observations acc over epoch: 0.00015
total loss: 1.10693, total regularisd loss: 0.45248
obs u loss: 0.21757, pde u loss: 0.00071
lambda obs u: 1.07937, lambda pde u: 0.00086
obs v loss: 0.08814, pde v loss: 0.06835
lambda obs v: 0.42421, lambda pde v: 2.49556
sigma_a:0:0.00020, sigma_s:0:0.01747, mu_a:0:0.01004, rho_a:0:0.01042, rho_s:0:0.01746, kappa_a:0:0.28530
Time taken: 26.79s

Start of epoch 462
Training observations acc over epoch: 0.00015
total loss: 1.26118, total regularisd loss: 0.43664
obs u loss: 0.21339, pde u loss: 0.00070
lambda obs u: 1.07024, lambda p

Training observations acc over epoch: 0.00014
total loss: 1.14734, total regularisd loss: 0.40139
obs u loss: 0.19965, pde u loss: 0.00070
lambda obs u: 1.02774, lambda pde u: 0.00091
obs v loss: 0.08686, pde v loss: 0.06477
lambda obs v: 0.46116, lambda pde v: 2.51019
sigma_a:0:0.00014, sigma_s:0:0.01726, mu_a:0:0.01027, rho_a:0:0.01050, rho_s:0:0.01745, kappa_a:0:0.27890
Time taken: 26.87s

Start of epoch 481
Training observations acc over epoch: 0.00014
total loss: 1.17371, total regularisd loss: 0.40892
obs u loss: 0.20118, pde u loss: 0.00070
lambda obs u: 1.04432, lambda pde u: 0.00091
obs v loss: 0.08503, pde v loss: 0.06491
lambda obs v: 0.44679, lambda pde v: 2.50798
sigma_a:0:0.00003, sigma_s:0:0.01724, mu_a:0:0.01028, rho_a:0:0.01044, rho_s:0:0.01759, kappa_a:0:0.27915
Time taken: 26.82s

Start of epoch 482
Training observations acc over epoch: 0.00014
total loss: 1.16542, total regularisd loss: 0.40299
obs u loss: 0.19415, pde u loss: 0.00070
lambda obs u: 1.00842, lambda p

In [5]:
layers = [3, 64, 64, 64, 64, 2]

pinn2 = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss2 = ASDM_multi(dtype=tf.float64, print_precision="0.8f")

In [7]:
model2 = tu_multi.TINN_multi_nodes(pinn2, 
                                  pde_loss2,
                                  nodes_n = 2,
                                  node_names= ["u", "v"],
                                  alpha = 0.5,
                                  print_precision=".6f")

In [8]:
results2 = model2.train(epochs = 1000,
                      batch_size = 512,
                      X = obs_X,
                      Y = obs_Y,
                      print_interval=1,
                      stop_threshold = 1e-6,                                  
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 8.254646
total loss: 37105.102834, total regularisd loss: 37105.102834
obs u loss: 586.038979, pde u loss: 934.963796
lambda obs u: 1.000000, lambda pde u: 1.000000
obs v loss: 16319.486525, pde v loss: 19264.613534
lambda obs v: 1.000000, lambda pde v: 1.000000
D_a:0: 9.77501205, D_s:0: 11.37671125, sigma_a:0: 10.02510205, sigma_s:0: 9.21486265, mu_a:0: 9.84263775, rho_a:0: 10.64338667, rho_s:0: 11.04088974, kappa_a:0: 8.92302043
Time taken: 42.95s

Start of epoch 1
Training observations acc over epoch: 6.353144
total loss: 20344.961710, total regularisd loss: 20344.961710
obs u loss: 656.725231, pde u loss: 1.904300
lambda obs u: 0.535754, lambda pde u: 0.500001
obs v loss: 12354.507076, pde v loss: 7331.825104
lambda obs v: 1.976502, lambda pde v: 0.987744
D_a:0: 9.52350854, D_s:0: 12.24556309, sigma_a:0: 10.06882597, sigma_s:0: 8.43805314, mu_a:0: 9.68045201, rho_a:0: 11.08907067, rho_s:0: 11.86006724, kappa_a:0: 8.00513908
Ti

Training observations acc over epoch: 0.123748
total loss: 4745.415713, total regularisd loss: 820.317995
obs u loss: 207.981007, pde u loss: 287.364442
lambda obs u: 3.655014, lambda pde u: 0.138500
obs v loss: 45.454343, pde v loss: 125.861850
lambda obs v: 0.105784, lambda pde v: 0.100702
D_a:0: 9.27297558, D_s:0: 13.45941820, sigma_a:0: 7.34264485, sigma_s:0: 4.24820826, mu_a:0: 10.34518921, rho_a:0: 10.35665018, rho_s:0: 14.78590725, kappa_a:0: 3.25638141
Time taken: 26.72s

Start of epoch 18
Training observations acc over epoch: 0.120274
total loss: 3869.469738, total regularisd loss: 791.527175
obs u loss: 200.284429, pde u loss: 311.639734
lambda obs u: 3.585885, lambda pde u: 0.189917
obs v loss: 46.035941, pde v loss: 113.729937
lambda obs v: 0.108698, lambda pde v: 0.115501
D_a:0: 8.27355672, D_s:0: 12.26109628, sigma_a:0: 7.12522976, sigma_s:0: 4.19397611, mu_a:0: 10.19641558, rho_a:0: 10.51163530, rho_s:0: 14.41330926, kappa_a:0: 3.19337626
Time taken: 26.43s

Start of epo

Training observations acc over epoch: 0.039991
total loss: 363.247080, total regularisd loss: 167.633332
obs u loss: 32.310989, pde u loss: 86.406876
lambda obs u: 1.327241, lambda pde u: 0.530225
obs v loss: 49.591535, pde v loss: 35.625943
lambda obs v: 1.341405, lambda pde v: 0.801128
D_a:0: 0.00000000, D_s:0: 0.07835915, sigma_a:0: 0.00000000, sigma_s:0: 0.44910381, mu_a:0: 5.33117118, rho_a:0: 15.48250550, rho_s:0: 1.30610173, kappa_a:0: 2.30951912
Time taken: 26.74s

Start of epoch 35
Training observations acc over epoch: 0.038692
total loss: 199.969834, total regularisd loss: 151.289747
obs u loss: 33.667422, pde u loss: 65.838799
lambda obs u: 1.116638, lambda pde u: 0.627213
obs v loss: 45.574018, pde v loss: 13.184308
lambda obs v: 1.688586, lambda pde v: 0.567564
D_a:0: 0.00000000, D_s:0: 0.05448693, sigma_a:0: 0.00000000, sigma_s:0: 0.24404335, mu_a:0: 5.41908688, rho_a:0: 15.01351801, rho_s:0: 0.69134116, kappa_a:0: 2.21662676
Time taken: 26.24s

Start of epoch 36
Training

Training observations acc over epoch: 0.007982
total loss: 37.925931, total regularisd loss: 36.289140
obs u loss: 7.374586, pde u loss: 14.199861
lambda obs u: 1.269322, lambda pde u: 0.491350
obs v loss: 8.972326, pde v loss: 0.000000
lambda obs v: 2.239316, lambda pde v: 0.000012
D_a:0: 0.00000000, D_s:0: 0.00000335, sigma_a:0: 0.56579505, sigma_s:0: 0.00001419, mu_a:0: 7.09466319, rho_a:0: 10.59150380, rho_s:0: 0.00001809, kappa_a:0: 0.81691204
Time taken: 26.55s

Start of epoch 52
Training observations acc over epoch: 0.007314
total loss: 35.378769, total regularisd loss: 33.212977
obs u loss: 6.888443, pde u loss: 12.926206
lambda obs u: 1.184680, lambda pde u: 0.488772
obs v loss: 8.090877, pde v loss: 0.000000
lambda obs v: 2.326542, lambda pde v: 0.000006
D_a:0: 0.01302005, D_s:0: 0.00000290, sigma_a:0: 0.65324361, sigma_s:0: 0.00001224, mu_a:0: 7.25507659, rho_a:0: 10.33732761, rho_s:0: 0.00001464, kappa_a:0: 0.76406619
Time taken: 26.58s

Start of epoch 53
Training observati

Training observations acc over epoch: 0.002450
total loss: 228.244912, total regularisd loss: 11.776959
obs u loss: 2.946588, pde u loss: 6.213780
lambda obs u: 1.570643, lambda pde u: 0.515082
obs v loss: 2.070413, pde v loss: 0.000000
lambda obs v: 1.914275, lambda pde v: 0.000000
D_a:0: 0.31544977, D_s:0: 0.00000264, sigma_a:0: 1.42224901, sigma_s:0: 0.00000728, mu_a:0: 8.56059445, rho_a:0: 8.05385428, rho_s:0: 0.00000643, kappa_a:0: 0.35660024
Time taken: 26.64s

Start of epoch 69
Training observations acc over epoch: 0.002384
total loss: 236.839688, total regularisd loss: 11.480295
obs u loss: 2.878217, pde u loss: 6.061718
lambda obs u: 1.557912, lambda pde u: 0.530661
obs v loss: 2.004603, pde v loss: 0.000000
lambda obs v: 1.911427, lambda pde v: 0.000000
D_a:0: 0.31836675, D_s:0: 0.00000264, sigma_a:0: 1.43222387, sigma_s:0: 0.00000727, mu_a:0: 8.56534765, rho_a:0: 8.01172804, rho_s:0: 0.00000641, kappa_a:0: 0.35202436
Time taken: 27.65s

Start of epoch 70
Training observation

...

Training observations acc over epoch: 0.000108
total loss: 1.454315, total regularisd loss: 0.520863
obs u loss: 0.152535, pde u loss: 0.000002
lambda obs u: 2.916244, lambda pde u: 0.000005
obs v loss: 0.069274, pde v loss: 0.000000
lambda obs v: 1.083751, lambda pde v: 0.000000
D_a:0: 0.00001074, D_s:0: 0.00000010, sigma_a:0: 0.00001130, sigma_s:0: 0.00000334, mu_a:0: 0.00013463, rho_a:0: 0.00012560, rho_s:0: 0.00000284, kappa_a:0: 0.25437291
Time taken: 27.79s

Start of epoch 919
Training observations acc over epoch: 0.000116
total loss: 1.473914, total regularisd loss: 0.559283
obs u loss: 0.165210, pde u loss: 0.000002
lambda obs u: 2.938307, lambda pde u: 0.000005
obs v loss: 0.071501, pde v loss: 0.000000
lambda obs v: 1.061687, lambda pde v: 0.000000
D_a:0: 0.00001075, D_s:0: 0.00000009, sigma_a:0: 0.00001063, sigma_s:0: 0.00000336, mu_a:0: 0.00013567, rho_a:0: 0.00012447, rho_s:0: 0.00000283, kappa_a:0: 0.25437291
Time taken: 27.22s

Start of epoch 920
Training observations ac

Training observations acc over epoch: 0.000108
total loss: 1.455358, total regularisd loss: 0.522053
obs u loss: 0.153018, pde u loss: 0.000002
lambda obs u: 2.915090, lambda pde u: 0.000005
obs v loss: 0.068641, pde v loss: 0.000000
lambda obs v: 1.084905, lambda pde v: 0.000000
D_a:0: 0.00001098, D_s:0: 0.00000001, sigma_a:0: 0.00001096, sigma_s:0: 0.00000334, mu_a:0: 0.00013462, rho_a:0: 0.00012456, rho_s:0: 0.00000283, kappa_a:0: 0.25437287
Time taken: 27.09s

Start of epoch 936
Training observations acc over epoch: 0.000113
total loss: 1.442016, total regularisd loss: 0.543590
obs u loss: 0.160085, pde u loss: 0.000002
lambda obs u: 2.924797, lambda pde u: 0.000005
obs v loss: 0.070906, pde v loss: 0.000000
lambda obs v: 1.075198, lambda pde v: 0.000000
D_a:0: 0.00001043, D_s:0: 0.00000001, sigma_a:0: 0.00001084, sigma_s:0: 0.00000331, mu_a:0: 0.00013497, rho_a:0: 0.00012419, rho_s:0: 0.00000286, kappa_a:0: 0.25437287
Time taken: 26.69s

Start of epoch 937
Training observations ac

Training observations acc over epoch: 0.000106
total loss: 1.441259, total regularisd loss: 0.508669
obs u loss: 0.148723, pde u loss: 0.000002
lambda obs u: 2.918428, lambda pde u: 0.000005
obs v loss: 0.068080, pde v loss: 0.000000
lambda obs v: 1.081567, lambda pde v: 0.000000
D_a:0: 0.00001067, D_s:0: 0.00000000, sigma_a:0: 0.00001145, sigma_s:0: 0.00000329, mu_a:0: 0.00013420, rho_a:0: 0.00012381, rho_s:0: 0.00000288, kappa_a:0: 0.25437282
Time taken: 27.45s

Start of epoch 953
Training observations acc over epoch: 0.000107
total loss: 1.411462, total regularisd loss: 0.511226
obs u loss: 0.149876, pde u loss: 0.000002
lambda obs u: 2.923622, lambda pde u: 0.000005
obs v loss: 0.068256, pde v loss: 0.000000
lambda obs v: 1.076372, lambda pde v: 0.000000
D_a:0: 0.00001077, D_s:0: 0.00000000, sigma_a:0: 0.00001148, sigma_s:0: 0.00000332, mu_a:0: 0.00013422, rho_a:0: 0.00012364, rho_s:0: 0.00000284, kappa_a:0: 0.25437282
Time taken: 27.22s

Start of epoch 954
Training observations ac

Training observations acc over epoch: 0.000107
total loss: 1.391978, total regularisd loss: 0.515562
obs u loss: 0.151155, pde u loss: 0.000002
lambda obs u: 2.931377, lambda pde u: 0.000005
obs v loss: 0.067929, pde v loss: 0.000000
lambda obs v: 1.068618, lambda pde v: 0.000000
D_a:0: 0.00001070, D_s:0: 0.00000000, sigma_a:0: 0.00001084, sigma_s:0: 0.00000335, mu_a:0: 0.00013380, rho_a:0: 0.00012302, rho_s:0: 0.00000282, kappa_a:0: 0.25437277
Time taken: 27.07s

Start of epoch 970
Training observations acc over epoch: 0.000105
total loss: 1.422436, total regularisd loss: 0.505056
obs u loss: 0.148041, pde u loss: 0.000002
lambda obs u: 2.922772, lambda pde u: 0.000005
obs v loss: 0.066528, pde v loss: 0.000000
lambda obs v: 1.077222, lambda pde v: 0.000000
D_a:0: 0.00001058, D_s:0: 0.00000000, sigma_a:0: 0.00001098, sigma_s:0: 0.00000333, mu_a:0: 0.00013378, rho_a:0: 0.00012297, rho_s:0: 0.00000283, kappa_a:0: 0.25437277
Time taken: 26.79s

Start of epoch 971
Training observations ac

Training observations acc over epoch: 0.000109
total loss: 1.348788, total regularisd loss: 0.526690
obs u loss: 0.155289, pde u loss: 0.000002
lambda obs u: 2.945748, lambda pde u: 0.000005
obs v loss: 0.067041, pde v loss: 0.000000
lambda obs v: 1.054246, lambda pde v: 0.000000
D_a:0: 0.00001044, D_s:0: 0.00000000, sigma_a:0: 0.00001121, sigma_s:0: 0.00000331, mu_a:0: 0.00013307, rho_a:0: 0.00012271, rho_s:0: 0.00000285, kappa_a:0: 0.25437273
Time taken: 27.08s

Start of epoch 987
Training observations acc over epoch: 0.000108
total loss: 1.379856, total regularisd loss: 0.521758
obs u loss: 0.153122, pde u loss: 0.000002
lambda obs u: 2.951653, lambda pde u: 0.000005
obs v loss: 0.067061, pde v loss: 0.000000
lambda obs v: 1.048341, lambda pde v: 0.000000
D_a:0: 0.00001039, D_s:0: 0.00000000, sigma_a:0: 0.00001068, sigma_s:0: 0.00000336, mu_a:0: 0.00013318, rho_a:0: 0.00012260, rho_s:0: 0.00000278, kappa_a:0: 0.25437272
Time taken: 26.97s

Start of epoch 988
Training observations ac